## Step 1: Creating a SQLite3 Database from CSV

In this step, we will:
- Create a SQLite database using the `sqlite3` module
- Read the HR dataset CSV using `pandas`
- Store the dataset as a table inside the database

This will allow us to run SQL queries directly on the dataset in the following steps.


In [1]:
# Import required libraries
import pandas as pd
import sqlite3
import os


In [2]:
# Load the dataset
df = pd.read_csv('C:/Users/rohit/Downloads/WA_Fn-UseC_-HR-Employee-Attrition.csv')

# Preview the data
df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# Create a SQLite database and connection
db_path = 'hr_attrition.db'

# Remove existing db if already exists for clean rerun
if os.path.exists(db_path):
    os.remove(db_path)

# Connect to the new database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [4]:
# Store the dataframe as a table in the SQLite database
df.to_sql('hr_data', conn, index=False, if_exists='replace')

# Confirm table creation
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Tables in database:", tables)


Tables in database: [('hr_data',)]


### ✅ SQLite3 DB Created Successfully

The CSV data has been loaded into a SQLite database named `hr_attrition.db`.  
The table `hr_data` contains the entire HR dataset and is now ready for SQL queries in the next steps.


## Step 2: Attrition Rate and Group-wise Summarization

In this step, we will calculate the attrition rate across the entire dataset and then break it down by different categories using SQL:
- Gender
- Department
- Age
- Job Level (with Average Monthly Income)
- Years at Company

This will help us identify patterns and demographics where attrition is higher.


In [5]:
query = """
SELECT 
    Attrition,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM hr_data) AS Attrition_rate
FROM hr_data
GROUP BY Attrition;
"""
# Convert 'Yes'/'No' to True/False for display match
df_attrition_rate = pd.read_sql_query(query, conn)
df_attrition_rate['Attrition'] = df_attrition_rate['Attrition'].map({'Yes': True, 'No': False})
df_attrition_rate['Attrition_rate'] = df_attrition_rate['Attrition_rate'].round(1).astype(str) + '%'
df_attrition_rate


,Attrition,Attrition_rate
0,False,83.9%
1,True,16.1%


In [14]:
query = """
SELECT 
    Attrition,
    Gender,
    COUNT(*) AS Count_gender,
    ROUND(COUNT(*) * 100.0 / (
        SELECT COUNT(*) FROM hr_data WHERE Gender = hr.AttrGender
    ), 1) AS Attrition_by_gender
FROM (
    SELECT *, Gender AS AttrGender FROM hr_data
) hr
GROUP BY Gender,Attrition
Order By Attrition Desc;
"""
df_gender = pd.read_sql_query(query, conn)
df_gender['Attrition'] = df_gender['Attrition'].map({'Yes': True, 'No': False})
df_gender


,Attrition,Gender,Count_gender,Attrition_by_gender
0,True,Female,87,14.8
1,True,Male,150,17.0
2,False,Female,501,85.2
3,False,Male,732,83.0


In [16]:
query = """
SELECT 
    Department,
    Attrition,
    COUNT(*) AS Department_attrition
FROM hr_data
GROUP BY Department,Attrition;
"""
pd.read_sql_query(query, conn)


,Department,Attrition,Department_attrition
0,Human Resources,No,51
1,Human Resources,Yes,12
2,Research & Development,No,828
3,Research & Development,Yes,133
4,Sales,No,354
5,Sales,Yes,92


In [12]:
query = """
SELECT 
    Attrition,
    CASE
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 40 THEN '30 - 40'
        WHEN Age BETWEEN 41 AND 50 THEN '40 -50'
        ELSE 'Over 50'
    END AS Age_group,
    COUNT(*) AS num,
    ROUND(COUNT(*) * 100.0 / (
        SELECT COUNT(*) FROM hr_data WHERE 
        CASE
            WHEN Age < 30 THEN 'Under 30'
            WHEN Age BETWEEN 30 AND 40 THEN '30 - 40'
            WHEN Age BETWEEN 41 AND 50 THEN '40 -50'
            ELSE 'Over 50'
        END = Age_group
    ), 2) AS percent_by_age
FROM (
    SELECT *,
        CASE
            WHEN Age < 30 THEN 'Under 30'
            WHEN Age BETWEEN 30 AND 40 THEN '30 - 40'
            WHEN Age BETWEEN 41 AND 50 THEN '40 -50'
            ELSE 'Over 50'
        END AS Age_group
    FROM hr_data
)
GROUP BY Attrition, Age_group
ORDER BY Attrition, Age_group;
"""
df_age = pd.read_sql_query(query, conn)
df_age['Attrition'] = df_age['Attrition'].map({'Yes': True, 'No': False})
df_age


,Attrition,Age_group,num,percent_by_age
0,False,30 - 40,585,86.16
1,False,40 -50,288,89.44
2,False,Over 50,125,87.41
3,False,Under 30,235,72.09
4,True,30 - 40,94,13.84
5,True,40 -50,34,10.56
6,True,Over 50,18,12.59
7,True,Under 30,91,27.91


In [17]:
query = """
SELECT 
    Department,
    JobLevel,
    ROUND(AVG(MonthlyIncome), 1) AS avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome END), 1) AS attrition_avg_income,
    ROUND(
        ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome END), 1)
        - ROUND(AVG(MonthlyIncome), 1), 1
    ) AS difference
FROM hr_data
GROUP BY Department, JobLevel
HAVING difference  IS NOT NULL
ORDER BY Department, JobLevel;
"""
pd.read_sql_query(query, conn)


,Department,JobLevel,avg_income,attrition_avg_income,difference
0,Human Resources,1,2733.2,2415.7,-317.5
1,Human Resources,3,9623.0,10216.0,593.0
2,Research & Development,1,2840.1,2687.4,-152.7
3,Research & Development,2,5291.2,5372.0,80.8
4,Research & Development,3,10170.5,9503.8,-666.7
5,Research & Development,4,15634.7,12169.0,-3465.7
6,Research & Development,5,19218.5,19550.0,331.5
7,Sales,1,2506.7,2373.4,-133.3
8,Sales,2,5746.1,5917.0,170.9
9,Sales,3,9282.3,9202.8,-79.5


In [21]:
query = """
SELECT 
    CASE
        WHEN YearsAtCompany <= 1 THEN 'New Hires'
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2-5 years'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 years'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 years'
        ELSE 'Over 20 years'
    END AS tenure_years,
    COUNT(*) AS num,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM hr_data WHERE Attrition = 'Yes'), 1) AS percent
FROM hr_data
WHERE Attrition = 'Yes'
GROUP BY tenure_years
ORDER BY Num Desc;
"""
pd.read_sql_query(query, conn)


,tenure_years,num,percent
0,2-5 years,87,36.7
1,New Hires,75,31.6
2,6-10 years,55,23.2
3,11-20 years,12,5.1
4,Over 20 years,8,3.4


## Step 3: Exploring Main Reasons for Attrition Using SQL

In this step, we will go beyond simple groupings and try to answer specific business questions using SQL. This helps us derive insights that can inform strategic HR decisions:
- Why do people over 50 leave more than those aged 40–50?
- Why do high-paid employees still leave?
- Which factors drive employees with <5 years at the company to leave?


In [27]:
query = """
SELECT 
    CASE 
        WHEN Age BETWEEN 41 AND 50 THEN '40-50'
        WHEN Age > 50 THEN 'Over 50'
    END AS Age_Group,
    JobRole,
    WorkLifeBalance,
    DistanceFromHome,
    MonthlyIncome,
    PercentSalaryHike
    JobInvolvement
FROM hr_data
WHERE Attrition = 'Yes' AND Age > 40
ORDER BY Age_Group Asc, WorkLifeBalance ASC, DistanceFromHome DESC;
"""
pd.read_sql_query(query, conn)


,Age_Group,JobRole,WorkLifeBalance,DistanceFromHome,MonthlyIncome,JobInvolvement
0,40-50,Laboratory Technician,1,24,3161,22
1,40-50,Sales Executive,1,9,12936,11
2,40-50,Research Scientist,1,5,2107,17
3,40-50,Sales Executive,1,1,5993,11
4,40-50,Manager,2,29,11849,12
5,40-50,Sales Representative,2,20,3140,22
6,40-50,Sales Executive,2,12,13758,12
7,40-50,Sales Executive,2,9,5346,13
8,40-50,Laboratory Technician,2,2,2778,13
9,40-50,Research Scientist,2,1,2342,12


In [32]:
query = """
SELECT 
    Department,
    JobRole,
    JobLevel,
    MonthlyIncome,
    OverTime,
    WorkLifeBalance,
    JobInvolvement,
    DistanceFromHome,
    PerformanceRating
FROM hr_data
WHERE Attrition = 'Yes' AND MonthlyIncome > 10000
ORDER BY MonthlyIncome DESC, OverTime DESC, WorkLifeBalance DESC;
"""
pd.read_sql_query(query, conn)


,Department,JobRole,JobLevel,MonthlyIncome,OverTime,WorkLifeBalance,JobInvolvement,DistanceFromHome,PerformanceRating
0,Research & Development,Manager,5,19859,Yes,3,3,2,3
1,Sales,Manager,5,19845,No,3,1,2,3
2,Research & Development,Research Director,5,19545,No,3,3,12,3
3,Research & Development,Research Director,5,19246,Yes,3,3,2,3
4,Sales,Manager,5,18824,Yes,3,1,2,3
5,Sales,Sales Executive,4,13758,Yes,2,3,12,3
6,Sales,Sales Executive,4,13695,Yes,2,4,13,3
7,Research & Development,Manager,3,13610,Yes,4,3,15,3
8,Sales,Sales Executive,4,13194,Yes,2,4,24,3
9,Sales,Sales Executive,4,12936,No,1,1,9,3


In [33]:
query = """
SELECT 
    Age,
    Department,
    JobRole,
    YearsAtCompany,
    WorkLifeBalance,
    OverTime,
    JobInvolvement,
    EnvironmentSatisfaction,
    DistanceFromHome,
    MonthlyIncome
FROM hr_data
WHERE Attrition = 'Yes' AND YearsAtCompany < 5
ORDER BY YearsAtCompany
limit 20;
"""
pd.read_sql_query(query, conn)


,Age,Department,JobRole,YearsAtCompany,WorkLifeBalance,OverTime,JobInvolvement,EnvironmentSatisfaction,DistanceFromHome,MonthlyIncome
0,37,Research & Development,Laboratory Technician,0,3,Yes,2,4,2,2090
1,19,Sales,Sales Representative,0,2,Yes,3,4,22,1675
2,19,Sales,Sales Representative,0,4,No,1,3,1,2325
3,28,Research & Development,Laboratory Technician,0,1,No,3,1,2,3485
4,18,Research & Development,Laboratory Technician,0,3,No,3,3,3,1420
5,18,Sales,Sales Representative,0,3,Yes,3,2,5,1878
6,23,Research & Development,Laboratory Technician,0,3,Yes,4,3,6,1601
7,29,Research & Development,Research Scientist,0,3,Yes,2,4,10,2404
8,50,Sales,Sales Executive,0,3,Yes,3,4,1,4728
9,18,Research & Development,Laboratory Technician,0,3,No,3,3,8,1904


### 🔍 Summary of Findings 

- **Over 50 vs 40–50:** Employees above 50 may experience lower job involvement or work-life balance, which leads to higher attrition even though they are more experienced.
- **High-Paid Leavers:** High salary doesn’t guarantee retention — poor environment, high overtime, or lack of job satisfaction may be stronger drivers.
- **<5 Years at Company:** Employees in their early tenure are highly vulnerable to attrition, especially if they experience long commutes, high pressure, lower saleries or lack of support.

These insights reinforce that compensation is **not the only** factor; satisfaction, culture, and balance matter just as much.